In [ ]:
!pip install -q unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
from unsloth import FastLanguageModel
import torch

MAX_SEQ_LENGTH = 2048
BASE_MODEL = "unsloth/Meta-Llama-3.1-70B-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(model_name=BASE_MODEL,
                                                     device_map="auto",
                                                     load_in_4bit=True)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.11.9: Fast Gemma2 patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

# Add LoRA Adapters

In [ ]:
model = FastLanguageModel.get_peft_model(model, r=32,
                                         target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                                                          "gate_proj", "up_proj", "down_proj",],
                                         lora_alpha=64,
                                         lora_dropout=0,
                                         bias="none",
                                         use_gradient_checkpointing="unsloth",
                                         use_rslora=False,
                                         loftq_config=None
        )

Unsloth 2024.11.9 patched 42 layers with 42 QKV layers, 42 O layers and 42 MLP layers.


# Data Preprocessing

In [ ]:
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

In [ ]:
dataset = load_dataset("csv", data_files="/content/AlpacaCodeFormat.csv", delimiter=",", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
def formatting_prompts_func(example):
    output_texts = []
    min_len = min(len(example["prompt"]), len(example["input"]), len(example["output"]))
    for i in range(min_len):
        text = f"### Question: {example['prompt'][i]}\n### Input: {example['input'][i]}\n### Answer: {example['output'][i]}"
        output_texts.append(text)
    return output_texts

response_template = "### Answer:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset["train"],
    formatting_func=formatting_prompts_func,
    max_seq_length=MAX_SEQ_LENGTH,
    data_collator=collator,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
      per_device_train_batch_size=8,
      gradient_accumulation_steps=4,
      warmup_steps=10,
      max_steps=100,
      learning_rate=2e-4,
      fp16=not is_bfloat16_supported(),
      bf16=is_bfloat16_supported(),
      optim="adamw_8bit",
      weight_decay=0.01,
      lr_scheduler_type="linear",
      report_to="none",
      output_dir="outputs",
      save_total_limit=2,
    )

)

Map (num_proc=2):   0%|          | 0/30 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 30 | Num Epochs = 8
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 15
 "-____-"     Number of trainable parameters = 216,072,192


Step,Training Loss


# Inference the Model

In [ ]:
prompt = """
### Question: Generate a set of interview questions for a Teknisi CCTV & Jaringan position at CV Media Hamparan Langit. The questions should be derived from the following job description and candidate requirements.
    Job Description:
    Deskripsi pekerjaan DevSecOps PT Solusi Identitas Global NetResponsible for installing hardware (servers) and software, including virtualization, operating systems, and applications required for PSrE operations. Configure and optimize to enhance the performance of physical and virtual servers.Manage and administer user access according to the access control matrix. Monitor and report on server and container capacity management.Maintain all PSrE servers, both physical and virtual. Plan, ensure, and monitor the operational availability of all PSrE systems.Perform, monitor, document, and ensure backup processes and their results. Conduct periodic restore tests to verify the usability and integrity of backups.Manage patch updates based on Vulnerability Assessments. Address incidents with corrective and preventive actions to ensure service functionality and achieve established RTO and RPO.Document all tasks and outcomes in the ticketing system. Prepare periodic written reports on progress, design, and other information required by PSrE management.Ensure information security (confidentiality, integrity, availability) in all tasks performed, in compliance with regulations and ISO 27001. Lihat Lebih Sedikit
### Input: Nama: Eva Morgan
    Asal Institusi: Universitas Gadjah Mada
    Prodi: Psikologi
### Answer:
"""

FastLanguageModel.for_inference(model)
inputs = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(
    **inputs,
    max_length=2048,
    num_return_sequences=1,
    do_sample=True,
    temperature=0.7,
)

# Decode the generated output
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)



### Question: Generate a set of interview questions for a Teknisi CCTV & Jaringan position at CV Media Hamparan Langit. The questions should be derived from the following job description and candidate requirements.
    Job Description:
    Deskripsi pekerjaan DevSecOps PT Solusi Identitas Global NetResponsible for installing hardware (servers) and software, including virtualization, operating systems, and applications required for PSrE operations. Configure and optimize to enhance the performance of physical and virtual servers.Manage and administer user access according to the access control matrix. Monitor and report on server and container capacity management.Maintain all PSrE servers, both physical and virtual. Plan, ensure, and monitor the operational availability of all PSrE systems.Perform, monitor, document, and ensure backup processes and their results. Conduct periodic restore tests to verify the usability and integrity of backups.Manage patch updates based on Vulnerability 

# Save Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model.save_pretrained("intersim-model-v1.1.0")
tokenizer.save_pretrained("intersim-model-v1.1.0")

# compress to zip
# !zip -r intersim-model-v1.1.0.zip intersim-model-v1
# upload to drive
# !cp intersim-model-v1.zip /content/drive/MyDrive/Bangkit-Nyoh/models

('intersim-model-v1.1.0/tokenizer_config.json',
 'intersim-model-v1.1.0/special_tokens_map.json',
 'intersim-model-v1.1.0/tokenizer.model',
 'intersim-model-v1.1.0/added_tokens.json',
 'intersim-model-v1.1.0/tokenizer.json')